# BACKPROPAGATION

In [1]:
import pandas as pd
import tensorflow as tf

In [16]:
df = pd.read_csv('preprocessed_1.csv')
df = df.dropna()
df = pd.get_dummies(df, columns=['Location', 'WindGustDir', 'WindDir9am', 'WindDir3pm', 'RainToday'])
df['Date'] = pd.to_datetime(df['Date'])
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df = df.drop(['Date'], axis=1)

In [3]:
# Split the data into training and testing sets
train_size = int(len(df) * 0.8)
train_df = df[:train_size]
test_df = df[train_size:]

In [4]:
# Separate the target variable from the input variables
X_train = train_df.drop('RainTomorrow', axis=1)
y_train = train_df['RainTomorrow']
X_test = test_df.drop('RainTomorrow', axis=1)
y_test = test_df['RainTomorrow']

In [5]:
from keras.layers import Dense,Flatten
# Define the neural network architecture
bpn_model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(16, input_dim=X_train.shape[1], activation='relu'),
  tf.keras.layers.Dense(8, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
]) 


In [6]:
bpn_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [7]:
bpn_model.fit(X_train, y_train, epochs=100, batch_size=32)

Epoch 1/100
3637/3637 [==============================] - 13s 3ms/step - loss: 0.7231 - accuracy: 0.8043
Epoch 2/100
3637/3637 [==============================] - 11s 3ms/step - loss: 0.4031 - accuracy: 0.8256
Epoch 3/100
3637/3637 [==============================] - 10s 3ms/step - loss: 0.3949 - accuracy: 0.8292
Epoch 4/100
3637/3637 [==============================] - 10s 3ms/step - loss: 0.3921 - accuracy: 0.8314
Epoch 5/100
3637/3637 [==============================] - 10s 3ms/step - loss: 0.3875 - accuracy: 0.8325
Epoch 6/100
3637/3637 [==============================] - 10s 3ms/step - loss: 0.3869 - accuracy: 0.8333
Epoch 7/100
3637/3637 [==============================] - 10s 3ms/step - loss: 0.3846 - accuracy: 0.8345
Epoch 8/100
3637/3637 [==============================] - 11s 3ms/step - loss: 0.3833 - accuracy: 0.8346
Epoch 9/100
3637/3637 [==============================] - 10s 3ms/step - loss: 0.3823 - accuracy: 0.8350 0s - loss: 0.3825 - accuracy
Epoch 10/100
3637/3637 [===========

In [8]:
# Evaluate the model on the testing data
loss, accuracy = bpn_model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')

910/910 [==============================] - 2s 2ms/step - loss: 0.3303 - accuracy: 0.8553
Test accuracy: 0.8552523255348206


In [9]:
from keras.models import load_model
bpn_model.save('./models/backpropagation.h5')

# RNN

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics

In [2]:
import keras
from keras.models import Sequential

In [3]:
df = pd.read_csv("preprocessed_1.csv")
X = df.drop(["RainTomorrow", "Date"], axis=1)
Y = df["RainTomorrow"]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state = 0)

In [5]:
y_train

43447     0
140904    0
54615     0
109793    0
74462     0
         ..
106567    0
52791     1
56339     0
82299     1
115175    0
Name: RainTomorrow, Length: 116368, dtype: int64

In [6]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 90866, 1: 25502})
The number of classes after fit Counter({0: 90866, 1: 90866})


In [7]:
import tensorflow as tf

X_train_res = tf.expand_dims(X_train_res, axis=1)

In [8]:
X_test = tf.expand_dims(X_test, axis=1)

In [9]:
# reshape your labels as 2d-tensor
y_train_res = np.asarray(y_train_res).astype('float32').reshape((-1,1))
y_test = np.asarray(y_test).astype('float32').reshape((-1,1))

In [10]:
num_features=X.shape[1]
num_features

23

In [11]:
import numpy as np
import keras
from keras.models import Sequential
from keras.layers import Dense,Flatten,SimpleRNN,Input

# define the input shape and number of classes
num_classes = 1

# create a sequential model
rnn_model = Sequential()

rnn_model.add(Input(shape=(None,num_features)))
# add the first dense layer with ReLU activation
rnn_model.add(SimpleRNN(32, return_sequences=True))
rnn_model.add(SimpleRNN(16))

# add the output layer with sigmoid activation
rnn_model.add(Dense(num_classes, activation='sigmoid'))

# Check here
rnn_model.add(Flatten())

# compile the model, specifying the optimizer, loss function, and metrics
rnn_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# fit the model to the training data
rnn_model.fit(X_train_res, y_train_res, epochs=100, batch_size=64)

# evaluate the model on the test data
test_loss, test_acc = rnn_model.evaluate(X_test, y_test)
print('Test accuracy:', test_acc)


Epoch 1/100
2840/2840 [==============================] - 16s 5ms/step - loss: 0.5743 - accuracy: 0.7294
Epoch 2/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5468 - accuracy: 0.7559
Epoch 3/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5451 - accuracy: 0.7595
Epoch 4/100
2840/2840 [==============================] - 14s 5ms/step - loss: 0.5408 - accuracy: 0.7619
Epoch 5/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5420 - accuracy: 0.7614
Epoch 6/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5405 - accuracy: 0.7625
Epoch 7/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5388 - accuracy: 0.7641
Epoch 8/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5383 - accuracy: 0.7638
Epoch 9/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5353 - accuracy: 0.7648
Epoch 10/100
2840/2840 [==============================] - 13s 5m

2840/2840 [==============================] - 13s 5ms/step - loss: 0.5049 - accuracy: 0.7713
Epoch 80/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5046 - accuracy: 0.7712
Epoch 81/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5049 - accuracy: 0.7715
Epoch 82/100
2840/2840 [==============================] - 14s 5ms/step - loss: 0.5037 - accuracy: 0.7716
Epoch 83/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5053 - accuracy: 0.7718
Epoch 84/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5058 - accuracy: 0.7711
Epoch 85/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5030 - accuracy: 0.7722
Epoch 86/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5028 - accuracy: 0.7717
Epoch 87/100
2840/2840 [==============================] - 13s 5ms/step - loss: 0.5028 - accuracy: 0.7714
Epoch 88/100
2840/2840 [==============================] - 13s 5ms/st

In [12]:
from keras.models import load_model
rnn_model.save('./models/rnn.h5')

# MLP

In [26]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
import joblib
import pickle

In [27]:
df = pd.read_csv("preprocessed_1.csv")
pd.set_option("display.max_columns", None)
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date_month,Date_day
0,2008-12-01,30,13.4,22.9,0.6,2.4,8.3,4.0,44.0,5.0,3.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,0.0,16.9,21.8,0,0,12,1
1,2008-12-02,30,7.4,25.1,0.0,3.6,10.0,2.0,44.0,0.0,6.0,4.0,22.0,44.0,25.0,1010.6,1007.8,7.0,1.0,17.2,24.3,0,0,12,2
2,2008-12-03,30,12.9,25.7,0.0,2.6,4.4,5.0,46.0,5.0,6.0,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,0,0,12,3
3,2008-12-04,30,9.2,28.0,0.0,14.6,8.9,11.0,24.0,13.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,0.0,5.0,18.1,26.5,0,0,12,4
4,2008-12-05,30,17.5,32.3,1.0,5.4,3.0,4.0,41.0,12.0,0.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0,12,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,49,2.8,23.4,0.0,1.4,7.8,15.0,31.0,13.0,13.0,13.0,11.0,51.0,24.0,1024.6,1020.3,1.0,1.0,10.1,22.4,0,0,6,21
145456,2017-06-22,49,3.6,25.3,0.0,7.6,13.5,0.0,22.0,13.0,2.0,13.0,9.0,56.0,21.0,1023.5,1019.1,8.0,5.0,10.9,24.5,0,0,6,22
145457,2017-06-23,49,5.4,26.9,0.0,6.8,11.0,3.0,37.0,13.0,3.0,9.0,9.0,53.0,24.0,1021.0,1016.8,8.0,1.0,12.5,26.1,0,0,6,23
145458,2017-06-24,49,7.8,27.0,0.0,2.6,13.2,12.0,28.0,11.0,2.0,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,0,0,6,24


In [28]:
X = df.drop(["RainTomorrow", "Date"], axis=1)
Y = df["RainTomorrow"]

In [29]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state = 0)

In [30]:
y_train

43447     0
140904    0
54615     0
109793    0
74462     0
         ..
106567    0
52791     1
56339     0
82299     1
115175    0
Name: RainTomorrow, Length: 116368, dtype: int64

In [31]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 90866, 1: 25502})
The number of classes after fit Counter({0: 90866, 1: 90866})


In [32]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train_res)  
X_train_res = scaler.transform(X_train_res)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [33]:
from sklearn.neural_network import MLPClassifier

clf = MLPClassifier(solver='adam', alpha=1e-5,
                    hidden_layer_sizes=(32,20), random_state=1)

clf.fit(X_train_res, y_train_res)

MLPClassifier(alpha=1e-05, hidden_layer_sizes=(32, 20), random_state=1)

In [34]:
y_pred = clf.predict(X_test)

In [35]:
print("Accuracy:", clf.score(X_test, y_test))

Accuracy: 0.8106008524680325


In [36]:
import pickle
with open('./models/mlp.pkl', 'wb') as file:
    pickle.dump(clf, file)

# RBF

In [37]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
import scipy.stats as stats
from sklearn.model_selection import train_test_split
from collections import Counter
from imblearn.over_sampling import SMOTE
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report
from sklearn import metrics
import joblib
import pickle

In [38]:
df = pd.read_csv("preprocessed_1.csv")
pd.set_option("display.max_columns", None)
df

,Date,Location,MinTemp,MaxTemp,Rainfall,Evaporation,Sunshine,WindGustDir,WindGustSpeed,WindDir9am,WindDir3pm,WindSpeed9am,WindSpeed3pm,Humidity9am,Humidity3pm,Pressure9am,Pressure3pm,Cloud9am,Cloud3pm,Temp9am,Temp3pm,RainToday,RainTomorrow,Date_month,Date_day
0,2008-12-01,30,13.4,22.9,0.6,2.4,8.3,4.0,44.0,5.0,3.0,20.0,24.0,71.0,22.0,1007.7,1007.1,8.0,0.0,16.9,21.8,0,0,12,1
1,2008-12-02,30,7.4,25.1,0.0,3.6,10.0,2.0,44.0,0.0,6.0,4.0,22.0,44.0,25.0,1010.6,1007.8,7.0,1.0,17.2,24.3,0,0,12,2
2,2008-12-03,30,12.9,25.7,0.0,2.6,4.4,5.0,46.0,5.0,6.0,19.0,26.0,38.0,30.0,1007.6,1008.7,8.0,2.0,21.0,23.2,0,0,12,3
3,2008-12-04,30,9.2,28.0,0.0,14.6,8.9,11.0,24.0,13.0,14.0,11.0,9.0,45.0,16.0,1017.6,1012.8,0.0,5.0,18.1,26.5,0,0,12,4
4,2008-12-05,30,17.5,32.3,1.0,5.4,3.0,4.0,41.0,12.0,0.0,7.0,20.0,82.0,33.0,1010.8,1006.0,7.0,8.0,17.8,29.7,0,0,12,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145455,2017-06-21,49,2.8,23.4,0.0,1.4,7.8,15.0,31.0,13.0,13.0,13.0,11.0,51.0,24.0,1024.6,1020.3,1.0,1.0,10.1,22.4,0,0,6,21
145456,2017-06-22,49,3.6,25.3,0.0,7.6,13.5,0.0,22.0,13.0,2.0,13.0,9.0,56.0,21.0,1023.5,1019.1,8.0,5.0,10.9,24.5,0,0,6,22
145457,2017-06-23,49,5.4,26.9,0.0,6.8,11.0,3.0,37.0,13.0,3.0,9.0,9.0,53.0,24.0,1021.0,1016.8,8.0,1.0,12.5,26.1,0,0,6,23
145458,2017-06-24,49,7.8,27.0,0.0,2.6,13.2,12.0,28.0,11.0,2.0,13.0,7.0,51.0,24.0,1019.4,1016.5,3.0,2.0,15.1,26.0,0,0,6,24


In [39]:
X = df.drop(["RainTomorrow", "Date"], axis=1)
Y = df["RainTomorrow"]

In [40]:
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size =0.2, stratify = Y, random_state = 0)

In [41]:
y_train

43447     0
140904    0
54615     0
109793    0
74462     0
         ..
106567    0
52791     1
56339     0
82299     1
115175    0
Name: RainTomorrow, Length: 116368, dtype: int64

In [42]:
sm=SMOTE(random_state=0)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)
print("The number of classes before fit {}".format(Counter(y_train)))
print("The number of classes after fit {}".format(Counter(y_train_res)))

The number of classes before fit Counter({0: 90866, 1: 25502})
The number of classes after fit Counter({0: 90866, 1: 90866})


In [43]:
from sklearn.preprocessing import StandardScaler  
scaler = StandardScaler()  
# Don't cheat - fit only on training data
scaler.fit(X_train_res)  
X_train_res = scaler.transform(X_train_res)  
# apply same transformation to test data
X_test = scaler.transform(X_test) 

In [44]:
from sklearn.svm import SVC

In [45]:
# Train the radial basis function (RBF) network
rbf_model = SVC(kernel='rbf')
rbf_model.fit(X_train_res, y_train_res)

SVC()

In [46]:
import pickle
with open('./models/rbf.pkl', 'wb') as file:
    pickle.dump(clf, file)

In [47]:
print("Accuracy:", rbf_model.score(X_test, y_test))

Accuracy: 0.8161350199367524
